In [13]:
# create observed pairs from travel time
fsrcrec = "OUTPUT_FILES/src_rec_file_forward.dat"

# read line by line
with open(fsrcrec, "r") as f:
    lines = f.readlines()
    


In [14]:
# iterate over lines to get the line number of source definition

src_lines = []
n_recs = []

irec = 0

for i, line in enumerate(lines):
    # if line includes "src_"
    if "src_" in line:
        # get the line number
        src_lines.append(i)
        
        # get the number of receivers
        if (i != 0):
            n_recs.append(irec)
        irec=0

    else:
        irec += 1

# add the last number of receivers
n_recs.append(irec)



In [15]:
lines[src_lines[-1]]

'    499 1998 1 1 0 0  0.00   41.8400   26.8400    8.0000  3.00    30 src_499 1.0000\n'

In [16]:
n_recs[-1]

30

In [17]:
# create new src rec file with insertion of station pairs

fname = "OUTPUT_FILES/src_rec_file_forward_pairs.dat"

n_newlines = 0
lines_new = []

import numpy as np
def get_ids_closest_stations(rec_lines):
    # get the distance between stations
    dists = {}
    for i in range(len(rec_lines)):
        lon1=rec_lines[i].split()[3]
        lat1=rec_lines[i].split()[4]
        for j in range(i+1, len(rec_lines)):
            lon2 = rec_lines[j].split()[3]
            lat2 = rec_lines[j].split()[4]

            # epicentral distance between reci and recj
            dist = np.sqrt((float(lon1)-float(lon2))**2 + (float(lat1)-float(lat2))**2)

            # store the distance
            dists[(i,j)] = dist

    # get the two closest stations
    ist1, ist2 = min(dists, key=dists.get)

    return ist1, ist2

for isrc, i_src_line in enumerate(src_lines):
    # get the number of receivers
    nrec = n_recs[isrc]
    
    # get the source line
    src_line = lines[i_src_line]
    
    # get the receiver lines
    rec_lines = lines[i_src_line+1:i_src_line+nrec+1]

    # select the two closest stations
    ist1, ist2 = get_ids_closest_stations(rec_lines)

    # generate station pair using the last two stations (separating by white spaces)
    rec1 = rec_lines[ist1].split()
    rec2 = rec_lines[ist2].split()

    # generate pair line by concatenating rec1[0:6] + rec2[1:6] + rec1[6] + (float(rec1[7])-float(rec2[7]))
    pair_line = " ".join(rec1[0:6]) + " " + " ".join(rec2[1:6]) + " " + rec1[6] + " " + str(float(rec1[7])-float(rec2[7]))+"\n"

    # recompose src line
    src_new = src_line.split()
    
    # add +1 to the 11th column
    src_new[11] = str(int(src_new[11])+1)

    # rejoin the line
    src_new = " ".join(src_new)+"\n"

    # add new pair line to rec_lines
    rec_lines.append(pair_line)

    # add new srcs and recs to the new lines
    lines_new.append(src_new)
    lines_new.extend(rec_lines)

    n_newlines += 1



In [18]:
# write out lines_new

with open(fname, "w") as f:
    f.writelines(lines_new)